<img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/>

# DeepStream SDK 简介 #
DeepStream SDK 是一种流分析工具包，可用于构建视频 AI 应用。此开发套件可便于开发者结合使用现有或自定义插件，为其特定用例构建视频处理工作流，从而简化流程。借助 DeepStream，您可以更轻松地开始构建和部署基于 AI 的智能视频分析应用。

在开发智能视频分析解决方案时，DeepStream 可帮助用户处理以下繁重的任务：
* 利用硬件加速处理
* 优化工作流，实现高数据吞吐量和低延迟
* 优化神经网络模型，实现高速推理
* 同时处理多个视频流的数据
* 追踪视频每帧的相关元数据

在此过程中，我们可帮助开发者优先处理以下重要业务决策：
* 待分析视频流的种类和数量
* 待执行视频分析的类型
* AI 推理结果的后期处理

使用 DeepStream SDK，开发者可以专注于与项目目标和影响密切相关的更*重要*的任务。使用此工具包，开发者可以构建核心深度学习网络和 IP，而无需从头开始设计端到端解决方案。

## 学习目标 ##
使用此 Notebook，您将获得有效使用 NVIDIA DeepStream SDK 所需的基础知识，其中包括：
* GStreamer 和 DeepStream 的历史
* DeepStream 视频 AI 工作流剖析
* 不同类型的 DeepStream 插件
* 如何通过 DeepStream 工作流传输数据流

**目录**
<br>
本 Notebook 涵盖以下部分：
1. [视频 AI 应用示例](#s1)
    * [视频格式](#s11)
    * [练习 #1 - 运行示例应用](#e1)
2. [GStreamer 基础知识](#s2)
3. [DeepStream 工作流剖析](#s3) 
    * [查看插件](#s31)
    * [练习 #2 - 了解插件](#e2)
4. [获取通过 AI 推理生成的洞见](#s4)
    * [探针](#s41)

<a name='s1'></a>
## 视频 AI 应用示例 ##
我们来看一个视频 AI 应用示例。在本实验中，我们将构建 DeepStream 工作流来分析停车场摄像头输入画面。此示例应用使用的工作流与我们在下一个 Notebook 中构建的工作流相同。为便于演示，我们已将该过程重构为 [Python 脚本](sample_apps/app_2.py)。

<a name='s11'></a>
### 视频格式 ###
不同于我们熟悉的扩展名为 **.mp4** 的视频文件，输入视频文件是扩展名为 **.h264** 的编码视频文件。.mp4 文件扩展名是一种容器的表示形式，其中包含播放视频所需的所有文件。这些文件包括视觉图像、音轨和元数据（即比特率、分辨率、字幕、时间戳等）。元数据还包含有关音频和视频流所使用**编解码器**的信息。编解码（即*编码*和*解码*这两个词语的合成词）是一种为加快传输速度而将视频压缩（编码）成较小文件的方法。若要播放和处理编码文件，可以使用相同的编解码器对其解压缩（解码）。最常见的视频编解码器包括 **[H.264](https://en.wikipedia.org/wiki/Advanced_Video_Coding)**、**[H.265](https://en.wikipedia.org/wiki/High_Efficiency_Video_Coding)** 和 **[MPEG4](https://en.wikipedia.org/wiki/MPEG-4)**。**[MP4](https://en.wikipedia.org/wiki/MPEG-4_Part_14)** 与 MPEG4 不同，前者是一种容器，可用于在 JupyterLab 中进行播放。这些属性可对视频格式作出描述，我们还在不断开发新属性，以针对质量、文件大小和视频播放提供改善方案。我们需要根据输入和所需输出的视频格式来构建应用。

<p><img src='images/important.png' width=720></p>
在执行视频分析时，由于只需要视频组件，应用可能会使用 H.264 编码视频流，而非 MP4 容器文件。

<a name='e1'></a>
#### 练习 #1 - 运行示例应用 ####

**说明**：<br>
* 执行以下单元，将无法在 JupyterLab 中播放的 H.264 编码视频文件转换为 MP4 文件进行播放。
    * [FFmpeg](https://ffmpeg.org/) 工具是一款超快速的视频和音频转换器，使用通用语法：<br> `ffmpeg [global_options] {[input_file_options] -i input_url} ... {[output_file_options] output_url} ...`。<br> 使用 `ffmpeg` 命令时，`-i` 选项用于读取输入 URL，`-loglevel quiet` 选项用于屏蔽日志，`-y` 标志用于覆盖所有现有同名输出文件。
* 执行以下单元，查看经转换的输入视频。
* 执行以下单元，运行 DeepStream 工作流水线。由于我们设计了使用 MPEG4 编解码器编写编码输出文件的流水线，因此我们还将其转换为 MP4 容器文件以进行播放。
* 之后执行以下单元，查看输出视频。

In [ ]:
# DO NOT CHANGE THIS CELL
from IPython.display import Video

# Convert the H.264 encoded video file to MP4 container file - this will generate the sample_30.mp4 file
!ffmpeg -i /dli/task/data/sample_30.h264 /dli/task/sample_30.mp4 \
        -y \
        -loglevel quiet

# View the input video
Video('sample_30.mp4', width=720)

In [ ]:
# DO NOT CHANGE THIS CELL
# Run the DeepStream pipeline - this will generate the output_02_encoded.mpeg4 file
%run sample_apps/app_2.py data/sample_30.h264

In [ ]:
# DO NOT CHANGE THIS CELL
# Convert the encoded video file for playback - this will generate the output_02.mp4 file
!ffmpeg -i /dli/task/output_02_encoded.mpeg4 /dli/task/output_02.mp4 \
        -y \
        -loglevel quiet

# View the output video
Video('output_02.mp4', width=720)

<a name='s2'></a>
## GStreamer 基础知识 ##

DeepStream 可利用使用开源 [GStreamer 多媒体框架](https://gstreamer.freedesktop.org/)构建的优化图形架构。GStreamer 可用于创建多种流媒体应用，包括简单的媒体播放器和复杂的视频编辑应用。在任意工作流中均可组合搭配 GStreamer 插件，以创建自定义应用。

在构建应用之前，我们需要了解 GStreamer 中的一些重要概念。理解这些术语及其在软件中的作用，有助于我们理顺语法，以便使用 GStreamer 和 DeepStream。
* **元件** - 元件是 GStreamer 的核心。元件可在与其他元件关联时提供某种功能。例如，源元件提供流数据，过滤器元件处理数据流，而接收元件使用数据。从源元件发出的数据流会经过过滤器元件透传，流向下游的接收元件。默认情况下，GStreamer 提供大量元件，但也允许用户编写新元件。
    * 计算领域的[汇点(sink)](https://en.wikipedia.org/wiki/Sink_(computing))专为接收数据而设计。
* **Bin** - Bin 是容器元件，可用于将关联的元件组合成逻辑组。对 Bin 的处理方式与对任何其他元件的处理方式相同。您可对此元件执行编程，来管理其中包含的元件（包括状态更改及总线消息），以确保数据顺畅流动。在构建需要许多元件的复杂工作流时，此元件很有用。
* **工作流** - 作为顶层 Bin，工作流同样管理着所含元件的同步和总线消息。
* **插件** - 需要将元件封装在插件中，以便 GStreamer 使用。插件实质上是一种可加载的代码块，通常被视为共享对象文件或动态链接库。插件可能包含多个元件的实现，也可能只包含一个元件的实现。GStreamer 以插件形式提供基础模组，您可使用插件构建高效的视频分析工作流。DeepStream SDK 具有硬件加速插件，这些插件可将深度神经网络和其他复杂的处理任务引入视频流的处理流水线。
* **总线** - 总线是负责向应用传递元件所生成**消息**的对象。默认情况下，每个工作流都包含一个总线，因此应用只需参照对象的信号处理器，在总线上设置一个类似的消息处理器即可。系统会在主循环运行时定期检查总线是否有新消息，并在出现新消息时调用消息处理器。
    * 消息可表示应用的工作流活动。消息类型包括 `GST_MESSAGE_EOS`（流结束）、`GST_MESSAGE_ERROR` 和 `GST_MESSAGE_WARNING` 等。
* **衬垫 (Pad)** - 衬垫可用于协商 GStreamer 元件间的链路和数据流。作为元件上的“端口”，衬垫 (Pad) 可以连接其他元件，以便数据流的传输。当数据在管道中从一个元件流向另一个元件时，实际上是从一个元件的源衬垫 (Pad) 流向另一个元件的接收衬垫 (Pad)。只有在数据类型或**功能**兼容时，才能在两个衬垫 (Pad) 之间建立连接。
* **缓冲区**和**事件** - GStreamer 中的所有流均被分割成块，并被系统以下面两种 `GstMiniObject` 类型之一的形式从一个元件的源衬垫 (Pad) 传递到另一个元件的接收衬垫 (Pad)：**事件**（控制）和**缓冲区**（内容）。缓冲区是 GStreamer 中数据传输的基本单元。通常，其中包含从一个元件流向另一个元件的视频数据块。DeepStream SDK 可将 DeepStream 元数据对象 `NvDsBatchMeta` 附加到缓冲区。另一方面，事件则包含有关数据流在两个链接衬垫 (Pad) 之间流动的状态信息。事件可用于指示媒体流的结束。
* **查询** - 查询用于获取有关流的信息。

<p><img src='images/important.png' width=720></p>

大多数情况下，GStreamer 中的所有数据都是通过元件之间的链路进行单向流动。当数据从一个 DeepStream 元素流向另一个元素时，系统不会重新创建缓冲区，但会传递缓冲区指针，以避免不必要的复制，并实现高速性能。

<p><img src='images/gstreamer.png' width='720px'></p>

如需更多信息，请参阅 [GStreamer 基础知识](https://gstreamer.freedesktop.org/documentation/application-development/basics/index.html)。

<a name='s3'></a>
## DeepStream 工作流剖析 ##

GStreamer 和 DeepStream 应用均采用**基于插件的体系架构**。开发者可以通过封装元件的插件与这些元件交互。单个**插件**可能包含多个元件的实现，也可能只包含一个元件的实现。插件可执行特定功能，其创建目的就是方便开发者使用。构建工作流水线时，我们可以从可用的 [GStreamer](https://gstreamer.freedesktop.org/documentation/plugins_doc.html) 或 [DeepStream](https://docs.nvidia.com/metropolis/deepstream/dev-guide/text/DS_plugin_Intro.html#) 插件目录中选择，也可以创建新插件。应用可被视为由各个组件（插件）组成的工作流，每个组件都代表一个功能块，比如视频解码/编码、缩放、推理等功能块。

下图显示了典型的视频分析应用的工作流水线，涵盖从使用输入视频到输出对内容的理解（洞见）的全过程。各个功能块都是现实中使用的各种插件。底部是整个应用中使用的不同硬件引擎。在适用的情况下，插件使用底层硬件加速以提供最高性能。这可能涉及使用插件之间的零内存复制实现优化内存管理，以及使用各种加速器以确保呈现超高性能。
<p><img src='images/deepstream_overview_graph_architecture.png' width='720px'></p>

* 流数据可以通过 RTSP、本地文件系统或直接通过摄像头在网络中传输。这些流是使用 CPU 捕获的。将帧存储在内存中后，系统会使用 NVDEC 加速器发送这些帧以供解码。
* 解码后，可执行*可选的*图像预处理步骤，以便在推理前预处理输入图像。预处理可以是图像扭曲矫正或色彩空间转换。`Gst-nvdewarper` 插件可以对鱼眼图像或 360 度全景摄像头图像进行扭曲矫正。`Gst-nvvideoconvert` 插件可以在帧上执行色彩格式转换操作。这些插件都使用了 GPU 或 VIC（视觉图像合成器）。
* 下一步是对帧执行批量处理，实现卓越的推理性能。为此需使用 `Gst-nvstreammux` 插件。
* 对帧执行批量处理后，可发送这些帧以供推理。推理时可以使用 TensorRT（NVIDIA 的推理加速器运行时），也可以在 TensorFlow 或 PyTorch 等原生框架中使用 Triton 推理服务器。可使用 `Gst-nvinfer` 插件执行 TensorRT 推理，使用 `Gst-nvinferserver` 插件完成应用 Triton 的推理。
* 推理后，下一步是追踪物体。SDK 中配备了多个内置基准追踪器，其中包括高性能和高准确度追踪器。可使用 `Gst-nvtracker` 插件追踪物体。
* 为创建可视化辅件（如边界框、分割遮罩、标签），可使用名为 `Gst-nvdsosd` 的可视化插件。
* 最后，DeepStream 提供了用于输出结果的各种选项：使用屏幕上的边界框渲染输出、将输出保存到本地磁盘、通过 RTSP 流式输出或将元数据发送到云端。为将元数据发送到云端，DeepStream 会使用 `Gst-nvmsgconv` 和 `Gst-nvmsgbroker` 插件。`Gst-nvmsgconv` 可将元数据转换为模式载荷，而 `Gst-nvmsgbroker` 可连接到云端并发送遥测数据。DeepStream 还支持多个内置代理协议，如 Kafka、MQTT、AMQP 和 Azure IoT。您可以创建自定义代理适配器。

通过将不同插件连接至工作流水线，我们可以为自定义用例构建复杂的应用。

<a name='s31'></a>
### 查看插件 ### 
我们可以使用 `gst-inspect-1.0` 查看插件。此工具可以输出可用插件的信息、特定插件的信息或特定元件的信息。如果在没有*插件*或*元件*参数的情况下执行操作，则此工具会输出所有插件和元件的列表以及摘要。使用*插件*或*元件*参数执行操作时，此工具会输出有关该插件或元件的信息。

In [ ]:
# DO NOT CHANGE THIS CELL
!gst-inspect-1.0

现有大量插件可供开发者使用。如需了解更多相关信息，请参阅 [GStreamer 插件](https://gstreamer.freedesktop.org/documentation/plugins_doc.html) 和 [DeepStream 插件](https://docs.nvidia.com/metropolis/deepstream/dev-guide/text/DS_plugin_Intro.html#) 文档。现在不妨一览特定插件以了解详情。

In [ ]:
# DO NOT CHANGE THIS CELL
!gst-inspect-1.0 h264parse

我们获得了许多有用信息，但现在先关注*说明* 。通过查看 `h264parse` 插件，我们可以看到此插件专用于解析 H.264 视频流。为实现高效处理，视频数据通常以编码形式进行流式传输。我们通常使用 [H.264](https://en.wikipedia.org/wiki/H.264/MPEG-4_AVC) 执行压缩和编码，但也可使用其他选项，比如 H.265、VC1 和 MPEG-2 等等。压缩可以减少从一个位置到另一个位置的数据传输量，从而有助于加快处理速度。构建工作流水线时，如果我们需要解析 H.264 视频流，便可使用此插件。

<a name='e2'></a>
#### 练习 #2 - 了解插件 ####
查看 DeepStream 特定插件：`nvinfer`。

**说明**：<br>
* 仅在执行之前通过更改 `<FIXME>` 来修改以下单元。

In [ ]:
!gst-inspect-1.0 <<<<FIXME>>>>

点击 ... 以显示**答案**。

`nvinfer` 插件使用 NVIDIA TensorRT 对输入数据执行推理。此插件可以根据我们提供的训练模型，对（批量）图像进行 AI 推理，以执行分类、物体检测和分割任务。与推理引擎相关的多个属性均可支持设置，`model-engine-file` 属性便是其中之一。建议通过 `config-file-path` 属性的配置文件设置属性。如需有关 DeepStream 插件的更多信息，请参阅 [DeepStream 插件指南](https://docs.nvidia.com/metropolis/deepstream/dev-guide/index.html#plugins-development-guide)。

<a name='s4'></a>
## 访问 DeepStream 元数据 ##
`GstBuffer` 是 GStreamer 中数据传输的基本单元。通过工作流水线传递数据时，系统会将每个组件接收的元数据附加到缓冲区。同样，DeepStream SDK 会将 DeepStream 元数据对象 `NvDsBatchMeta` 附加到缓冲区。DeepStream 元数据包含 `Gst-nvinfer` 提供的推理结果以及工作流水线中其它插件提供的信息。从 `Gst-nvstreammux` 插件中创建的批量级元数据 (`NvDsBatchMeta`) 开始，DeepStream 就为其使用可扩展的标准结构。辅助元数据结构包含帧、对象、分类器和显示数据。如需元数据格式详情，请参阅 [SDK 元数据文档和 API 指南](https://docs.nvidia.com/metropolis/deepstream/dev-guide/text/DS_plugin_metadata.html)。熟悉元数据结构有助于提取所需信息。

<a name='s41'></a>
### 探针 ###
<p><img src='images/probe.png' width=720></p>

我们使用[探针](https://gstreamer.freedesktop.org/documentation/application-development/advanced/pipeline-manipulation.html#using-probes)访问此元数据。不妨将“用探针探查”这一过程视为获取衬垫 (Pad) 侦听器的相关信息。我们可以使用探针访问工作流水线中各节点的元数据。从技术角度讲，探针是可附加于衬垫 (Pad) 之上的[回调函数](https://en.wikipedia.org/wiki/Callback_(computer_programming))。附加后，探针会在衬垫 (Pad) 上发生数据传递时发出通知。探针使我们能够轻松地与流经我们管道的数据进行交互。如需 `GstPad` 和探针的更多信息，请访问 [GstPad](https://gstreamer.freedesktop.org/documentation/gstreamer/gstpad.html?gi-language=c) 上的 GStreamer API 参考资料。

<p><img src='images/important.png' width=720></p>

由于视频 AI 应用高度依赖深度学习模型所生成的元数据，因此探针回调函数可能是构建 DeepStream 工作流的重中之重。

**您做的不错！**准备就绪后，我们开始学习[下一个 Notebook](./03_building_a_DeepStream_application.ipynb)。

<img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/>